In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
import pandas as pd
import plotly.io as pio

# Sources
Infectious Period of Coronavirus: 10 days[^1]

Incubation period of Coronavirus: 5 days[^2]

US Population : 330,000,000[^3]

[^1]: https://www.cdc.gov/coronavirus/2019-ncov/hcp/duration-isolation.html?CDC_AA_refVal=https%3A%2F%2Fwww.cdc.gov%2Fcoronavirus%2F2019-ncov%2Fcommunity%2Fstrategy-discontinue-isolation.html
[^2]: https://www.acpjournals.org/doi/10.7326/M20-0504
[^3] : https://www.census.gov/popclock/

In [ ]:
def seir_model(t, y, beta, gamma, sigma, n):
    """ Solves system using the solve_ivp definition and function
    
    Arguments
        y: List of variable solutions
        t: List of time steps to evaluate at
        beta: Infection rate constant
        gamma: Recovery rate constant
        sigma: Latent activation rate constant
        n: Total number in the population
    
    Returns:
        ds: Change in S 
        di: Change in I
        dr: Change in R
        
    """
    s, e, i, r = y
    ds = - beta * s * i / n
    de = beta * s * i / n - sigma * e
    di = sigma * e - gamma * i
    dr = gamma * i

    return ds, de, di, dr

In [ ]:
### Starting Values
num_tsteps = 365*2
t = np.linspace(0,365,num_tsteps)
n = 330000000
y0 = [n - 1, 0, 1, 0] # Start with a single infected individual

gamma = 0.1 # 10 Day Infectious Period
beta = 5 * gamma # R0 = 5
sigma = 0.2 # 5-day latent period

infectious_time = round(1/gamma,2)
latent_time = round(1/sigma,2)

### Define steps for animation
beta_start = 0.01
beta_stop = 1
beta_step = 0.01
num_beta = int((beta_stop - beta_start)/beta_step + 1)
beta_list = np.linspace(beta_start, beta_stop, num_beta)

### DataFrame Initialization
column_names = ['beta', 'S', 'E', 'I', 'R']
data = pd.DataFrame(columns= column_names)

In [ ]:
### Pre-generate all data
for beta in beta_list:
    seir_solver = solve_ivp(seir_model, [0, 365], y0, args=(beta, gamma, sigma, n), dense_output=True)
    seir_solution = seir_solver.sol(t)
    temp_df = pd.DataFrame(list(zip([beta] * num_tsteps, seir_solution[0,:], seir_solution[1,:], seir_solution[2,:], seir_solution[3,:])), columns=column_names)
    data = pd.concat([data, temp_df])

In [ ]:
def gen_trace_list(column, trace_name, trace_color):
    trace = [go.Scatter(x=t, y=data[data['beta'] == beta][column],
                        visible=False, name = trace_name,
                        line_color = trace_color)
             for beta in data['beta'].unique()]
    return trace

s_list = gen_trace_list('S', 'Susceptible', 'red')
e_list = gen_trace_list('E', 'Exposed', 'green')
i_list = gen_trace_list('I', 'Infected', 'blue')
r_list = gen_trace_list('R', 'Recovered', 'goldenrod')

starting_beta = np.where(beta_list == 0.5)[0][0]
s_list[starting_beta]['visible'] = True
e_list[starting_beta]['visible'] = True
i_list[starting_beta]['visible'] = True
r_list[starting_beta]['visible'] = True

In [ ]:
# Add all of the data to a figure
fig = go.Figure(s_list + e_list + i_list + r_list)

# Generating all of the steps
steps = []
for i in range(num_beta):
    # Define the steps
    step = dict(
        method = 'update',
        label =  str(round(beta_list[i], 2)),
        args = [{'visible': [False] * len(fig.data)},
                {"title" : "SEIR Model – 1/𝛾 = 10 Days | 1/σ = 5 Days"}]
    )
    
    # Update the visible traces for each step
    step['args'][0]['visible'][i] = True
    step['args'][0]['visible'][i+num_beta] = True
    step['args'][0]['visible'][i+2*num_beta] = True
    step['args'][0]['visible'][i+3*num_beta] = True
    
    steps.append(step)

# Generate slider
sliders = [dict(steps = steps,
                active = starting_beta,
                currentvalue={'prefix' : "β = "},
                pad = {"t" : 50})]

# Final Figure generation
fig.update_layout(sliders=sliders,
                  title = "SEIR Model – 1/𝛾 = 10 Days | 1/σ = 5 Days",
                  template='plotly_white')
fig.update_yaxes(title='Number of Individuals')
fig.update_xaxes(title='Days')

# Figure preview
fig.show()

In [ ]:
# Write figure to HTML
pio.write_html(fig, "seir-graph.html", auto_open=False)